
improvements:

* clean up the client names by finding and merging any that are actually the same client under different names
* split out the mass and codes from the products

In [1]:
from fastai.tabular import *
from sklearn import preprocessing

In [2]:
data_path = Path("/home/jupyter/data/bimbo")

In [3]:
data_path.ls()

[PosixPath('/home/jupyter/data/bimbo/test.csv'),
 PosixPath('/home/jupyter/data/bimbo/zips'),
 PosixPath('/home/jupyter/data/bimbo/producto_tabla.csv'),
 PosixPath('/home/jupyter/data/bimbo/cliente_tabla.csv'),
 PosixPath('/home/jupyter/data/bimbo/sample_submission.csv'),
 PosixPath('/home/jupyter/data/bimbo/town_state.csv'),
 PosixPath('/home/jupyter/data/bimbo/train.csv')]

In [4]:
df_geo = pd.read_csv(data_path/'town_state.csv')
df_client = pd.read_csv(data_path/'cliente_tabla.csv')
df_product = pd.read_csv(data_path/'producto_tabla.csv')

os.makedirs('tmp', exist_ok=True)

In [5]:
if os.path.isfile('tmp/train_raw'):
    print("reading feather")
    df = pd.read_feather('tmp/train_raw')
else:
    print("reading train.csv")
    df = pd.read_csv(data_path/'train.csv')
    df.to_feather('tmp/train_raw')

reading feather


In [6]:
df.sample(10)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
73125336,9,3212,1,1429,222526,41843,8,54.88,0,0.00,8
38517234,6,1636,1,1201,290871,35651,2,15.00,0,0.00,2
19719490,4,2055,1,1051,477784,1150,6,83.76,0,0.00,6
19522280,4,2038,1,1111,47639,1238,2,19.66,0,0.00,2
4811752,3,1470,1,1152,1133179,48579,5,20.30,0,0.00,5
12453393,4,1153,1,1003,1566486,73,1,21.32,0,0.00,1
57356933,8,1423,1,2017,871884,35119,2,26.48,0,0.00,2
65731825,9,1224,1,1605,1736876,303,1,4.54,0,0.00,1
8685732,3,2045,1,5010,7781743,31720,2,16.76,1,8.38,1
11522293,4,1118,1,1402,1892685,1220,2,15.28,0,0.00,2


In [7]:
df_product.sample(10)

,Producto_ID,NombreProducto
558,31123,Lonchera Plastica Baul 284g CU GBI 31123
1059,34044,Bigotes Chocolate 60g Prom TR 34044
102,1198,Tostado 2pq 420g BIM 1198
455,30505,Tortillinas 22p 570g Prom TR 30505
1760,36944,Mini Milk Tarta Fresa 110g GBI 36944
591,31277,Rebanada 4p 110g BIM 31277
99,1182,Tostado 210g CU BIM 1182
610,31339,Lors 8p 74g SP TAB MLA 31339
977,33265,Charola Nito 15p 930g MTA BIM 33265
1066,34067,Lonchibox Prem Bco con Chile 90g LON 34067


In [8]:
df_client.sample(10)

,Cliente_ID,NombreCliente
193724,518114,DONA UVI
48397,116927,T V MEXIQUENSE
288890,951169,MIRIAM
871683,4710410,MANZUR
618847,2487387,MARIA
731076,4434807,NO IDENTIFICADO
320588,1111986,GUZMAN
39952,99226,ESC CBTA
584489,2371426,EDUARDO LOPEZ BARBA
429339,1703850,CITY TORTAS


In [9]:
df_geo.sample(10)

,Agencia_ID,Town,State
736,24619,2461 HIDALGO BIMBO,NUEVO LEÓN
85,1311,2273 SAN PEDRO,JALISCO
160,1448,2505 TORREON,COAHUILA
27,1152,2008 AG. LAGO FILT,"MÉXICO, D.F."
190,1492,2468 MATAMOROS BIMBO,TAMAULIPAS
37,1168,2092 AG. SANTA CLARA AUTOSERVICIOS,ESTADO DE MÉXICO
534,4082,2493 MITRAS,NUEVO LEÓN
740,24679,2467 LINCOLN,NUEVO LEÓN
358,1997,2573 TIJUANA GATO BRONCO,BAJA CALIFORNIA NORTE
413,2086,2165 LEON SAN JUAN BOSCO,GUANAJUATO


In [10]:
print(len((df_geo['Town'] + ", " + df_geo['State']).unique()))
print(len(df_geo))

261
790


In [11]:
print(len(df_client['NombreCliente'].unique()))
print(len(df_client))

311155
935362


In [12]:
print(len(df_product['NombreProducto'].unique()))
print(len(df_product))

2592
2592


In [13]:
df_geo['geo'] = (df_geo['Town'] + ", " + df_geo['State'])
le_geo = preprocessing.LabelEncoder()
df_geo['geo'] = le_geo.fit_transform(df_geo['geo'])
df_geo.to_feather('tmp/geo_map')

In [14]:
le_client = preprocessing.LabelEncoder()
df_client['NombreCliente'] = le_client.fit_transform(df_client['NombreCliente'])
df_client.to_feather('tmp/client_map')

In [15]:
le_product = preprocessing.LabelEncoder()
df_product['NombreProducto'] = le_product.fit_transform(df_product['NombreProducto'])
df_product.to_feather('tmp/product_map')

In [16]:
if os.path.isfile('tmp/df_merge'):
    print("read feather")
    df = pd.read_feather('tmp/df_merge')
else:
    print("merge raw data")
    df = (df.merge(df_geo[['Agencia_ID', 'geo']], how='left', on='Agencia_ID')
          .merge(df_product, how='left', on='Producto_ID')
          .merge(df_client, how='left', on='Cliente_ID')
         )
    df.to_feather('tmp/df_merge')

merge raw data


In [19]:
df.dtypes

Semana                 int64
Agencia_ID             int64
Canal_ID               int64
Ruta_SAK               int64
Cliente_ID             int64
Producto_ID            int64
Venta_uni_hoy          int64
Venta_hoy            float64
Dev_uni_proxima        int64
Dev_proxima          float64
Demanda_uni_equil      int64
geo                    int64
NombreProducto         int64
NombreCliente          int64
dtype: object